In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import r2_score

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# import the regressor
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Read the RF dataset from CSV file
data = pd.read_csv('RFLFSODataFull.csv')
print("Dataset:")
data.head(5)

Dataset:


,FSO_Att,RFL_Att,AbsoluteHumidity,AbsoluteHumidityMax,AbsoluteHumidityMin,Distance,Frequency,Particulate,ParticulateMax,ParticulateMin,...,TemperatureMax,TemperatureMin,Time,Visibility,VisibilityMax,VisibilityMin,WindDirection,WindSpeed,WindSpeedMax,WindSpeedMin
0,7.913289,6.927868,17.595709,17.615907,17.340148,2115.338398,83500000000,0.0,0.0,0.0,...,26.603030,24.680849,9,65884.51846,72336.362233,65617.543754,88.755450,3.057066,6.863808,3.007939
1,7.451176,4.412096,17.549693,17.572415,17.299439,2113.999257,73500000000,0.0,0.0,0.0,...,26.377164,24.313108,9,64963.41092,68753.386153,60379.327485,99.790057,2.727910,6.468903,2.537393
2,7.072747,6.268740,17.290230,17.644014,16.037894,2118.689047,83500000000,0.0,0.0,0.0,...,27.670822,23.150277,10,54794.28107,54821.773817,50850.155963,65.730085,1.674810,2.826916,1.640809
3,6.949288,4.317853,16.820880,17.066776,15.895622,2114.632339,73500000000,0.0,0.0,0.0,...,26.522100,23.174815,10,50205.64159,52519.927530,45374.510898,88.127497,0.962068,2.780643,0.886951
4,7.361052,6.114514,16.813820,17.953974,15.227225,2116.786055,83500000000,0.0,0.0,0.0,...,26.305736,24.864100,10,59038.32599,64418.329138,54461.246506,84.167414,1.881007,4.476298,1.874052


In [3]:
# Check the dimensions of the RF dataset
print("\nRF Dataset Dimensions:")
data.shape


RF Dataset Dimensions:


(91379, 27)

In [4]:
# Calculate summary statistics for the RF dataset
print("\nRF Dataset Summary Statistics:")
data.describe()


RF Dataset Summary Statistics:


,FSO_Att,RFL_Att,AbsoluteHumidity,AbsoluteHumidityMax,AbsoluteHumidityMin,Distance,Frequency,Particulate,ParticulateMax,ParticulateMin,...,TemperatureMax,TemperatureMin,Time,Visibility,VisibilityMax,VisibilityMin,WindDirection,WindSpeed,WindSpeedMax,WindSpeedMin
count,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,9.137900e+04,91379.000000,91379.000000,91379.000000,...,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000
mean,6.769458,11.619098,9.553919,10.032760,9.076251,3297.930328,7.850005e+10,27.065979,28.417120,25.717089,...,13.166020,11.904558,11.373193,32986.417484,34636.684647,31331.618978,89.043304,0.739065,1.851020,0.702242
std,3.903843,3.438873,5.858577,6.162798,5.575927,1224.305893,5.000027e+09,72.134023,75.761896,68.595239,...,9.589755,8.694028,6.942229,24713.874445,25978.845228,23497.873065,26.415610,0.871314,1.762698,0.828796
min,0.788363,0.027142,1.141556,1.238270,1.049744,2012.000148,7.350000e+10,0.000000,0.000000,0.000000,...,-6.693373,-7.270075,0.000000,10.151990,11.029667,9.484346,0.000000,0.000000,0.000000,0.000000
25%,3.473063,10.829331,4.958993,5.205861,4.709511,2019.431812,7.350000e+10,0.000000,0.000000,0.000000,...,4.905387,4.438425,5.000000,11158.598400,11721.447542,10588.793237,82.976727,0.076749,0.512667,0.072912
50%,6.336167,11.856560,6.870737,7.205499,6.524046,2959.863686,8.350000e+10,0.000000,0.000000,0.000000,...,10.803943,9.774467,11.000000,26378.636210,27686.214032,25047.188462,90.867362,0.437995,1.323101,0.416274
75%,8.664984,12.847944,14.049470,14.782679,13.379256,4820.890157,8.350000e+10,16.947618,17.775980,16.038090,...,22.470382,20.322434,17.000000,53493.821285,56092.212376,50797.690402,98.482652,1.088621,2.681788,1.036574
max,32.455222,46.893150,24.790883,26.407305,24.268431,4827.999971,8.350000e+10,1621.001906,1753.747866,1500.666382,...,40.771383,36.724191,23.000000,75004.999150,82503.130084,74999.343414,360.000000,7.799086,16.421115,7.252319


# Predict FSO

In [5]:
fso_data = data.drop("RFL_Att", axis = 1)

In [6]:
# Calculate summary statistics for the RF dataset
print("\nRFL Dataset Summary Statistics:")
fso_data.describe()


RFL Dataset Summary Statistics:


,FSO_Att,AbsoluteHumidity,AbsoluteHumidityMax,AbsoluteHumidityMin,Distance,Frequency,Particulate,ParticulateMax,ParticulateMin,RainIntensity,...,TemperatureMax,TemperatureMin,Time,Visibility,VisibilityMax,VisibilityMin,WindDirection,WindSpeed,WindSpeedMax,WindSpeedMin
count,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,9.137900e+04,91379.000000,91379.000000,91379.000000,91379.000000,...,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000
mean,6.769458,9.553919,10.032760,9.076251,3297.930328,7.850005e+10,27.065979,28.417120,25.717089,0.246973,...,13.166020,11.904558,11.373193,32986.417484,34636.684647,31331.618978,89.043304,0.739065,1.851020,0.702242
std,3.903843,5.858577,6.162798,5.575927,1224.305893,5.000027e+09,72.134023,75.761896,68.595239,1.642323,...,9.589755,8.694028,6.942229,24713.874445,25978.845228,23497.873065,26.415610,0.871314,1.762698,0.828796
min,0.788363,1.141556,1.238270,1.049744,2012.000148,7.350000e+10,0.000000,0.000000,0.000000,0.000000,...,-6.693373,-7.270075,0.000000,10.151990,11.029667,9.484346,0.000000,0.000000,0.000000,0.000000
25%,3.473063,4.958993,5.205861,4.709511,2019.431812,7.350000e+10,0.000000,0.000000,0.000000,0.000000,...,4.905387,4.438425,5.000000,11158.598400,11721.447542,10588.793237,82.976727,0.076749,0.512667,0.072912
50%,6.336167,6.870737,7.205499,6.524046,2959.863686,8.350000e+10,0.000000,0.000000,0.000000,0.000000,...,10.803943,9.774467,11.000000,26378.636210,27686.214032,25047.188462,90.867362,0.437995,1.323101,0.416274
75%,8.664984,14.049470,14.782679,13.379256,4820.890157,8.350000e+10,16.947618,17.775980,16.038090,0.000000,...,22.470382,20.322434,17.000000,53493.821285,56092.212376,50797.690402,98.482652,1.088621,2.681788,1.036574
max,32.455222,24.790883,26.407305,24.268431,4827.999971,8.350000e+10,1621.001906,1753.747866,1500.666382,87.264777,...,40.771383,36.724191,23.000000,75004.999150,82503.130084,74999.343414,360.000000,7.799086,16.421115,7.252319


In [7]:
# Handle categorical variable encoding
# Perform one-hot encoding for 'SYNOP Code'
fso_data = pd.get_dummies(fso_data, columns=['SYNOPCode'], prefix='SYNOP')

In [8]:
from sklearn.model_selection import train_test_split

# Split the dataset into features (X) and target variable (y)
X = fso_data.drop('FSO_Att', axis=1)
y = fso_data['FSO_Att']

# Split the data into training and testing sets
X_train, X_test, fso_y_train, fso_y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Create a Random Forest Regressor object
rf = RandomForestRegressor(n_estimators=300, max_depth = 40, random_state=42, n_jobs=-1)

# Fit the model on the training data
rf.fit(X_train, fso_y_train)

# Make predictions on the training and testing data
fso_train_predictions = rf.predict(X_train)
fso_test_predictions = rf.predict(X_test)

In [ ]:
fso_train_predictions

In [10]:
# Calculate the mean squared error
train_mse = mean_squared_error(fso_y_train, fso_train_predictions)
test_mse = mean_squared_error(fso_y_test, fso_test_predictions)

# Calculate root mean squared error  
train_rmse = math.sqrt(train_mse)

# Calculate root mean squared error 
test_rmse = math.sqrt(test_mse)

# Calculate the R-squared value
train_r2 = r2_score(fso_y_train, fso_train_predictions)
test_r2 = r2_score(fso_y_test, fso_test_predictions)

# Print the evaluation metrics
print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)
print("Train R^2:", train_r2)
print("Test R^2:", test_r2)

Train RMSE: 0.3095093478145377
Test RMSE: 0.8039965291046878
Train R^2: 0.9937614296237144
Test R^2: 0.9568086351547184


# Predict RFL Using FSO

In [11]:
# convert numpy array to dataframe 
df_predicted_train = pd.DataFrame(fso_train_predictions, columns =['FSO_Att'])

In [12]:
df_predicted_test = pd.DataFrame(fso_test_predictions, columns = ['FSO_Att'])

In [13]:
fso_predicted_data = pd.concat([df_predicted_train,df_predicted_test])

In [14]:
fso_predicted_data

,FSO_Att
0,8.756746
1,6.734001
2,2.849612
3,2.783834
4,8.248674
...,...
27409,15.741453
27410,17.283443
27411,2.284710
27412,5.585996


In [15]:
rfl_data = data.drop("FSO_Att", axis = 1).copy()

In [16]:
rfl_data

,RFL_Att,AbsoluteHumidity,AbsoluteHumidityMax,AbsoluteHumidityMin,Distance,Frequency,Particulate,ParticulateMax,ParticulateMin,RainIntensity,...,TemperatureMax,TemperatureMin,Time,Visibility,VisibilityMax,VisibilityMin,WindDirection,WindSpeed,WindSpeedMax,WindSpeedMin
0,6.927868,17.595709,17.615907,17.340148,2115.338398,83500000000,0.000000,0.000000,0.000000,0.0,...,26.603030,24.680849,9,65884.518460,72336.362233,65617.543754,88.755450,3.057066,6.863808,3.007939
1,4.412096,17.549693,17.572415,17.299439,2113.999257,73500000000,0.000000,0.000000,0.000000,0.0,...,26.377164,24.313108,9,64963.410920,68753.386153,60379.327485,99.790057,2.727910,6.468903,2.537393
2,6.268740,17.290230,17.644014,16.037894,2118.689047,83500000000,0.000000,0.000000,0.000000,0.0,...,27.670822,23.150277,10,54794.281070,54821.773817,50850.155963,65.730085,1.674810,2.826916,1.640809
3,4.317853,16.820880,17.066776,15.895622,2114.632339,73500000000,0.000000,0.000000,0.000000,0.0,...,26.522100,23.174815,10,50205.641590,52519.927530,45374.510898,88.127497,0.962068,2.780643,0.886951
4,6.114514,16.813820,17.953974,15.227225,2116.786055,83500000000,0.000000,0.000000,0.000000,0.0,...,26.305736,24.864100,10,59038.325990,64418.329138,54461.246506,84.167414,1.881007,4.476298,1.874052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91374,11.968291,23.283256,24.142871,22.169805,3970.846129,83500000000,0.000000,0.000000,0.000000,0.0,...,27.257926,26.013205,4,13041.847800,13458.340663,12411.852994,91.391749,0.000000,0.027596,0.000000
91375,11.538030,23.259618,25.104895,21.005496,3969.230652,73500000000,0.000000,0.000000,0.000000,0.0,...,28.101044,25.032282,4,15959.633590,16232.735439,15345.485232,47.997946,0.075640,0.349551,0.069834
91376,11.491891,23.331697,23.996130,22.148649,3968.373512,83500000000,0.000000,0.000000,0.000000,0.0,...,28.249678,26.205773,4,13129.663750,13948.560177,11925.979953,103.963102,0.052282,0.178062,0.049824
91377,11.526529,21.241027,21.624932,20.736391,3967.550165,73500000000,29.693160,30.761540,28.497137,0.0,...,26.880084,25.031934,5,15082.787860,15522.762221,14842.654291,82.090779,0.443520,1.838449,0.413146


In [17]:
# fso_data['RFL_Att'] = RFL_predicted_data
rfl_data.insert(1, "FSO_Att", fso_predicted_data, True)

In [18]:
print("\nRF Dataset Summary Statistics:")
rfl_data.describe()


RF Dataset Summary Statistics:


,RFL_Att,FSO_Att,AbsoluteHumidity,AbsoluteHumidityMax,AbsoluteHumidityMin,Distance,Frequency,Particulate,ParticulateMax,ParticulateMin,...,TemperatureMax,TemperatureMin,Time,Visibility,VisibilityMax,VisibilityMin,WindDirection,WindSpeed,WindSpeedMax,WindSpeedMin
count,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,9.137900e+04,91379.000000,91379.000000,91379.000000,...,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000,91379.000000
mean,11.619098,6.772549,9.553919,10.032760,9.076251,3297.930328,7.850005e+10,27.065979,28.417120,25.717089,...,13.166020,11.904558,11.373193,32986.417484,34636.684647,31331.618978,89.043304,0.739065,1.851020,0.702242
std,3.438873,3.800878,5.858577,6.162798,5.575927,1224.305893,5.000027e+09,72.134023,75.761896,68.595239,...,9.589755,8.694028,6.942229,24713.874445,25978.845228,23497.873065,26.415610,0.871314,1.762698,0.828796
min,0.027142,0.858784,1.141556,1.238270,1.049744,2012.000148,7.350000e+10,0.000000,0.000000,0.000000,...,-6.693373,-7.270075,0.000000,10.151990,11.029667,9.484346,0.000000,0.000000,0.000000,0.000000
25%,10.829331,3.519457,4.958993,5.205861,4.709511,2019.431812,7.350000e+10,0.000000,0.000000,0.000000,...,4.905387,4.438425,5.000000,11158.598400,11721.447542,10588.793237,82.976727,0.076749,0.512667,0.072912
50%,11.856560,6.401104,6.870737,7.205499,6.524046,2959.863686,8.350000e+10,0.000000,0.000000,0.000000,...,10.803943,9.774467,11.000000,26378.636210,27686.214032,25047.188462,90.867362,0.437995,1.323101,0.416274
75%,12.847944,8.682166,14.049470,14.782679,13.379256,4820.890157,8.350000e+10,16.947618,17.775980,16.038090,...,22.470382,20.322434,17.000000,53493.821285,56092.212376,50797.690402,98.482652,1.088621,2.681788,1.036574
max,46.893150,31.089854,24.790883,26.407305,24.268431,4827.999971,8.350000e+10,1621.001906,1753.747866,1500.666382,...,40.771383,36.724191,23.000000,75004.999150,82503.130084,74999.343414,360.000000,7.799086,16.421115,7.252319


In [19]:
# Handle categorical variable encoding
# Perform one-hot encoding for 'SYNOP Code'
rfl_data = pd.get_dummies(rfl_data, columns=['SYNOPCode'], prefix='SYNOP')

In [20]:
from sklearn.model_selection import train_test_split

# Split the dataset into features (X) and target variable (y)
X = rfl_data.drop('RFL_Att', axis=1)
y = rfl_data['RFL_Att']

# Split the data into training and testing sets
X_train, X_test, y_train, rfl_y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Create a Random Forest Regressor object
rf = RandomForestRegressor(n_estimators=300, max_depth = 40, random_state=42, n_jobs=-1)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Make predictions on the training and testing data
rfl_train_predictions = rf.predict(X_train)
rfl_test_predictions = rf.predict(X_test)

# Calculate the mean squared error
train_mse = mean_squared_error(y_train, rfl_train_predictions)
test_mse = mean_squared_error(rfl_y_test, rfl_test_predictions)

# Calculate root mean squared error  
train_rmse = math.sqrt(train_mse)

# Calculate root mean squared error 
test_rmse = math.sqrt(test_mse)

# Calculate the R-squared value
train_r2 = r2_score(y_train, rfl_train_predictions)
test_r2 = r2_score(rfl_y_test, rfl_test_predictions)

# Print the evaluation metrics
print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)
print("Train R^2:", train_r2)
print("Test R^2:", test_r2)

Train RMSE: 0.1981510079700676
Test RMSE: 0.5255751181868035
Train R^2: 0.9966907317040313
Test R^2: 0.9764593956166214


In [ ]:
from sklearn.metrics import mutual_info_score

# Calculate mutual information for training
mutual_information = mutual_info_score(rfl_train_predictions, fso_train_predictions)
print(mutual_information)

In [ ]:
from sklearn.metrics import mutual_info_score

# Calculate mutual information for training
mutual_information = mutual_info_score(rfl_test_predictions, fso_test_predictions)
print(mutual_information)

# Pearson Co-efficient 

In [ ]:
data = {
    'RFL_Att': rfl_train_predictions,
    'FSO_Att': fso_train_predictions
}

df = pd.DataFrame(data)

# Calculate the Pearson correlation coefficient
correlation = df['RFL_Att'].corr(df['FSO_Att'])

print("Pearson Correlation Coefficient:")
print(correlation)

In [ ]:
from scipy.stats import pearsonr
pear_corr = pearsonr(df['RFL_Att'], df['FSO_Att'])
pear_corr

In [ ]:
data = {
    'RFL_Att': rfl_test_predictions,
    'FSO_Att': fso_test_predictions
}

df = pd.DataFrame(data)

# Calculate the Pearson correlation coefficient
correlation = df['RFL_Att'].corr(df['FSO_Att'])

print("Pearson Correlation Coefficient:")
print(correlation)

In [ ]:
# For test
data = {
    'RFL_Att': rfl_y_test,
    "FSO_Att" : fso_y_test
}

df = pd.DataFrame(data)

# Calculate the Pearson correlation coefficient
correlation = df['RFL_Att'].corr(df['FSO_Att'])

print("Pearson Correlation Coefficient:")
print(correlation)